### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [14]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [37]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [38]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.50198881 0.52661709 0.5207708  0.52646471 0.53266128 0.47951529
 0.50555319 0.51697745 0.53001004 0.53975936 0.50272264 0.53490301
 0.52667049 0.52452885 0.53168112 0.49810331 0.49557109 0.50461987
 0.52069661 0.52287498 0.52362002 0.54528853 0.54434959 0.54284537
 0.54405258        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [68 60], Predictions: [0 1], [66 62]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [68 60], Predictions: [0 1], [65 63]
Training RF ...
Model RF, Actual: [0 1], [68 60], Predictions: [0 1], [74 54]
Training XGB ...
Model XGB, Actual: [0 1], [68 60], Predictions: [0 1], [67 61]

None


[('mean_SCL', 217), ('bpm', 202), ('rmssd', 149), ('hf_rr', 146), ('lf_rr', 142), ('sdnn', 141), ('SCR_rate', 139)]


[('mean_SCL', 0.16370090800770026), ('rmssd', 0.16019782643212327), ('bpm', 

One or more of the test scores are non-finite: [0.55447662 0.5607222  0.54859117 0.55213342 0.56131434 0.56734799
 0.54032321 0.54037237 0.54358195 0.54898112 0.56623461 0.56784052
 0.57365432 0.56514772 0.55500613 0.54911625 0.55723312 0.57648269
 0.56336882 0.56546312 0.58337513 0.55886721 0.5604612  0.56271293
 0.55772252        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [85 30], Predictions: [0 1], [82 33]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [85 30], Predictions: [0 1], [64 51]
Training RF ...
Model RF, Actual: [0 1], [85 30], Predictions: [0 1], [58 57]
Training XGB ...
Model XGB, Actual: [0 1], [85 30], Predictions: [0 1], [61 54]

None


[('mean_SCL', 121), ('bpm', 109), ('rmssd', 84), ('hf_rr', 78), ('lf_rr', 71), ('sdnn', 68), ('SCR_rate', 50)]


[('mean_SCL', 0.18982658920994436), ('bpm', 0.15485076007439078), ('rmssd', 0.

...

Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [72 31], Predictions: [0 1], [96  7]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [72 31], Predictions: [0 1], [77 26]
Training RF ...
Model RF, Actual: [0 1], [72 31], Predictions: [0 1], [80 23]
Training XGB ...
Model XGB, Actual: [0 1], [72 31], Predictions: [0 1], [83 20]

None


[('mean_SCL', 112), ('bpm', 104), ('rmssd', 102), ('lf_rr', 64), ('sdnn', 63), ('hf_rr', 59), ('SCR_rate', 47)]


[('rmssd', 0.16605590832537237), ('mean_SCL', 0.16423082272132802), ('lf_rr', 0.1618

One or more of the test scores are non-finite: [0.6107999  0.6094791  0.58846805 0.58530469 0.58986992 0.58485226
 0.61051356 0.60397707 0.60538455 0.60493212 0.53859505 0.56458592
 0.57440244 0.58225027 0.58868862 0.55204111 0.57799763 0.59532154
 0.59690425 0.60663636 0.54987456 0.57648869 0.57933862 0.59371485
 0.59232538        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [113  37], Predictions: [0 1], [85 65]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [113  37], Predictions: [0 1], [98 52]
Training RF ...
Model RF, Actual: [0 1], [113  37], Predictions: [0 1], [80 70]
Training XGB ...
Model XGB, Actual: [0 1], [113  37], Predictions: [0 1], [81 69]

None


[('bpm', 116), ('mean_SCL', 106), ('lf_rr', 77), ('hf_rr', 76), ('rmssd', 71), ('sdnn', 65), ('SCR_rate', 56)]


[('mean_SCL', 0.1758867745984783), ('bpm', 0.15564847647802657), ('rmssd',

One or more of the test scores are non-finite: [0.54782549 0.54740648 0.55455847 0.55977239 0.54577449 0.54716129
 0.53328093 0.55363634 0.56489233 0.55732516 0.51188059 0.52021162
 0.53821611 0.53683826 0.52608114 0.53791628 0.56208475 0.56848631
 0.5630196  0.55165901 0.56646094 0.56706878 0.57463207 0.5655896
 0.5636753         nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 30, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [65 54], Predictions: [0 1], [96 23]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [65 54], Predictions: [0 1], [83 36]
Training RF ...
Model RF, Actual: [0 1], [65 54], Predictions: [0 1], [82 37]
Training XGB ...
Model XGB, Actual: [0 1], [65 54], Predictions: [0 1], [86 33]

None


[('bpm', 210), ('mean_SCL', 195), ('rmssd', 178), ('lf_rr', 147), ('hf_rr', 142), ('SCR_rate', 133), ('sdnn', 129)]


[('mean_SCL', 0.15939934493109986), ('rmssd', 0.15904210983746137), ('bpm', 0.1507